# Article Link Parser

We scrape articles from a unreliable news source, and save it in a csv file.

In [ ]:
from bs4 import BeautifulSoup
import requests

from time import sleep
import pandas as pd

In [ ]:
all_links = []

soups = []

for n in range(1,295):
    r  = requests.get("http://www.medias-presse.info/category/politique/page/{}/".format(n))
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    
    soups.append(soup)
    sleep(0.2)

Now, we create a soup of all the pages existing in the political section of the website.

In [ ]:
for soup in soups:
    ls = []

    for link in soup.find_all('a'):
        url = link.get('href')
        x = url.split("/")
        if len(x) > 3:
            last_value = x[-2]
            last_value2 = x[-1]
            
            if last_value.isdigit():
                last_value = int(last_value)
                if last_value > 0 and last_value < 90000 and last_value2 != "#comments":
                    ls.append(url)
    
    all_links.extend(ls)
    print("Page {} done.".format(n))

all_links = set(all_links)
all_links = list(all_links)

Now, we parse the soup and only keep author, url, title and text in a dictionary.

In [ ]:
di_big = {"id":[], "agency": [], "url": [], "title": [], "text":[], "author":[]}
counter = 0

for link in all_links:
    sleep(0.1)
    r  = requests.get(link)
    data = r.text
    soup = BeautifulSoup(data, "lxml")

    paragraphs_ls = []

    for paragraph in soup.find_all('p', style="text-align: justify;"):
        paragraphs_ls.append(paragraph.text)

    paragraphs = "\n".join(paragraphs_ls)
    
    _id = link.split("/")[-2]
    author = soup.find('a', rel='author').text
    title = soup.title.text
    
    if paragraphs != "":
        di_big["id"].append(_id)
        di_big["agency"].append("media-presse.info")
        di_big["url"].append(link)
        di_big["title"].append(title)
        di_big["text"].append(paragraphs)
        di_big["author"].append(author)

In [ ]:
pd.DataFrame.from_dict(di_big).to_csv("fakenews_big.csv", index=False)